In [2]:
:dep tch = "0.13.0"

In [3]:
use tch::{nn, vision, Device};
use tch::nn::OptimizerConfig;
use tch::nn::Module;

In [11]:


const IMAGE_DIM: i64 = 784;
const LABELS: i64 = 10;

tch::manual_seed(42);
let device = Device::cuda_if_available();
let vs = nn::VarStore::new(device);

let cfg = nn::LinearConfig {
    ws_init: nn::Init::Const(0.),
    bs_init: Some(nn::Init::Const(0.)),
    bias: true,
};

let net = nn::seq()
    .add(nn::linear(
        &vs.root() / "layer1",
        IMAGE_DIM,
        LABELS,
        cfg,
    ));



In [12]:
let mut m = vision::mnist::load_dir("../data/mnist").unwrap();
m.train_images = m.train_images.to_device(device);
m.train_labels = m.train_labels.to_device(device);
m.test_images = m.test_images.to_device(device);
m.test_labels = m.test_labels.to_device(device);

In [13]:
let lr = 0.3;
let num_epochs = 10;
let batch_size = 500;

let mut opt: nn::Optimizer = nn::Sgd::default().build(&vs, lr).unwrap();

let mut loss_vec: Vec<f64> = Vec::new();

for epoch in 1..num_epochs {

   let loss = net.forward(&m.train_images,).cross_entropy_for_logits(&m.train_labels);
        opt.backward_step(&loss);

    let loss = net.forward(&m.train_images).cross_entropy_for_logits(&m.train_labels);
    let test_accuracy = net.forward(&m.test_images).accuracy_for_logits(&m.test_labels);
    loss_vec.push(f64::try_from(&loss).unwrap());
     println!(
        "epoch: {:4} train loss: {:8.5} test acc: {:5.2}%",
        epoch,
        f64::try_from(&loss).unwrap(),
        100. * f64::try_from(&test_accuracy).unwrap(),
    );
}

epoch:    1 train loss:  2.00253 test acc: 68.08%
epoch:    2 train loss:  1.76706 test acc: 76.13%
epoch:    3 train loss:  1.58109 test acc: 78.24%
epoch:    4 train loss:  1.43417 test acc: 79.67%
epoch:    5 train loss:  1.31711 test acc: 80.64%
epoch:    6 train loss:  1.22263 test acc: 81.35%
epoch:    7 train loss:  1.14524 test acc: 81.90%
epoch:    8 train loss:  1.08092 test acc: 82.49%
epoch:    9 train loss:  1.02672 test acc: 82.94%


()

Error: no method named `configure_mesh` found for mutable reference `&mut plotters::chart::ChartBuilder<'_, '_, SVGBackend<'_>>` in the current scope

Error: no method named `draw_series` found for mutable reference `&mut plotters::chart::ChartBuilder<'_, '_, SVGBackend<'_>>` in the current scope

Error: no method named `values` found for struct `std::ops::Range` in the current scope

Error: no method named `configure_series_labels` found for mutable reference `&mut plotters::chart::ChartBuilder<'_, '_, SVGBackend<'_>>` in the current scope

In [15]:
:dep polars = "0.30.0"

In [24]:
use polars::prelude::*;

let df = df! [
    "Model" => ["iPhone XS", "iPhone 12", "iPhone 13", "iPhone 14", "Samsung S11", "Samsung S12", "Mi A1", "Mi A2"],
    "Company" => ["Apple", "Apple", "Apple", "Apple", "Samsung", "Samsung", "Xiao Mi", "Xiao Mi"],
    "Sales" => [80, 170, 130, 205, 400, 30, 14, 8],
    "Comment" => [None, None, Some("Sold Out"), Some("New Arrival"), None, Some("Sold Out"), None, None],
]?;

println!("{}", &df); 

The type of the variable m was redefined, so was lost.
The type of the variable opt was redefined, so was lost.
The type of the variable net was redefined, so was lost.
The type of the variable cfg was redefined, so was lost.
The type of the variable vs was redefined, so was lost.
The type of the variable device was redefined, so was lost.


shape: (8, 4)
┌─────────────┬─────────┬───────┬─────────────┐
│ ---         ┆ ---     ┆ ---   ┆ ---         │
│ str         ┆ str     ┆ i32   ┆ str         │
│ Model       ┆ Company ┆ Sales ┆ Comment     │
╞═════════════╪═════════╪═══════╪═════════════╡
│ iPhone XS   ┆ Apple   ┆ 80    ┆ null        │
│ iPhone 12   ┆ Apple   ┆ 170   ┆ null        │
│ iPhone 13   ┆ Apple   ┆ 130   ┆ Sold Out    │
│ iPhone 14   ┆ Apple   ┆ 205   ┆ New Arrival │
│ Samsung S11 ┆ Samsung ┆ 400   ┆ null        │
│ Samsung S12 ┆ Samsung ┆ 30    ┆ Sold Out    │
│ Mi A1       ┆ Xiao Mi ┆ 14    ┆ null        │
│ Mi A2       ┆ Xiao Mi ┆ 8     ┆ null        │
└─────────────┴─────────┴───────┴─────────────┘


In [16]:
:dep plotters = { version = "0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

In [17]:
use plotters::prelude::*;

In [36]:
evcxr_figure((640, 480), |root| {
    let x_axis = 0..loss_vec.len();
    root.fill(&WHITE)?;
    let mut chart = ChartBuilder::on(&root)
        .margin(5)
        .set_all_label_area_size(50)
        .caption("loss", ("sans-serif", 20))
        .build_cartesian_2d(0f32..(loss_vec.len() as f32), 0f32..2f32)?;
    
    chart.configure_mesh()
        .x_labels(20)
        .y_labels(10)
        .disable_mesh()
        .draw()?;
    
    chart.draw_series(LineSeries::new(x_axis.map(|x| (x as f32, loss_vec[x] as f32)), &RED))?
        .label("loss")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));

    chart.configure_series_labels().border_style(&BLACK).draw()?;
    
    Ok(())
}).style("width:60%")

loss
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0
 
 
 
5.5
 
 
 
6.0
 
 
 
6.5
 
 
 
7.0
 
 
 
7.5
 
 
 
8.0
 
 
 
8.5
 
 
 
9.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0
 
 
 
5.5
 
 
 
6.0
 
 
 
6.5
 
 
 
7.0
 
 
 
7.5
 
 
 
8.0
 
 
 
8.5
 
 
 
9.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
 
 
loss